<a href="https://colab.research.google.com/github/MohanaThanmai/Electric-vehicle-charge-and-range-analysis/blob/main/Electric_vehicle_range_and_charge_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Setup of libraries**
For EV Charging station list

In [25]:
# Install any required libraries (Google Colab usually has pandas, numpy, matplotlib, seaborn pre-installed)
!pip install geopandas folium --quiet

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import geopandas as gpd

# Set some display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Setup complete ✅")


Setup complete ✅


**Data loading**

In [26]:
# If your CSV is in Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# Path to your dataset (adjust if needed)
file_path = '/content/electric_vehicle_charging_station_list.csv'

# Load data
df = pd.read_csv(file_path)

# Quick overview
print("Shape of dataset:", df.shape)
df.head()

Shape of dataset: (202, 8)


,region,address,aux addres,latitude,longitude,type,power,service
0,NDMC,"Prithviraj Market, Rabindra Nagar, New Delhi- ...","Electric Vehicle Charger, Prithviraj Market, R...",28.600725,77.226252,DC-001,15 kW,Self Service
1,NDMC,"Prithviraj Market, Rabindra Nagar, New Delhi- ...","Electric Vehicle Charger, Prithviraj Market, R...",28.600725,77.226252,DC-001,15 kW,Self Service
2,NDMC,"Outside RWA Park, Jor Bagh Market, Jor Bagh Co...","Electric Vehicle Charger, Outside RWA Park, Jo...",28.588303,77.217697,DC-001,15 kW,Self Service
3,NDMC,"Opposite Dory Pharmacy, Khanna Market, Aliganj...","Electric Vehicle Charger, Opposite Dory Pharma...",28.582654,77.220087,DC-001,15 kW,Self Service
4,NDMC,"Opposite Goel Opticals, Khanna Market, Aliganj...","Electric Vehicle Charger, Opposite Goel Optica...",28.584485,77.220316,DC-001,15 kW,Self Service


**Data preprocessing**

In [27]:
# Rename columns for consistency (lowercase, replace spaces with underscores)
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Check for missing values
print("Missing values per column:\n", df.isnull().sum())

# Data types
print("\nData types:\n", df.dtypes)

# Convert latitude & longitude to float
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

# Strip extra spaces in text columns
text_columns = ['region', 'address', 'aux_addres', 'type', 'power', 'service']
for col in text_columns:
    df[col] = df[col].astype(str).str.strip()

print("\nPreprocessing done ✅")
df.head()

Missing values per column:
 region        0
address       0
aux_addres    0
latitude      0
longitude     0
type          0
power         0
service       0
dtype: int64

Data types:
 region         object
address        object
aux_addres     object
latitude      float64
longitude     float64
type           object
power          object
service        object
dtype: object

Preprocessing done ✅


,region,address,aux_addres,latitude,longitude,type,power,service
0,NDMC,"Prithviraj Market, Rabindra Nagar, New Delhi- ...","Electric Vehicle Charger, Prithviraj Market, R...",28.600725,77.226252,DC-001,15 kW,Self Service
1,NDMC,"Prithviraj Market, Rabindra Nagar, New Delhi- ...","Electric Vehicle Charger, Prithviraj Market, R...",28.600725,77.226252,DC-001,15 kW,Self Service
2,NDMC,"Outside RWA Park, Jor Bagh Market, Jor Bagh Co...","Electric Vehicle Charger, Outside RWA Park, Jo...",28.588303,77.217697,DC-001,15 kW,Self Service
3,NDMC,"Opposite Dory Pharmacy, Khanna Market, Aliganj...","Electric Vehicle Charger, Opposite Dory Pharma...",28.582654,77.220087,DC-001,15 kW,Self Service
4,NDMC,"Opposite Goel Opticals, Khanna Market, Aliganj...","Electric Vehicle Charger, Opposite Goel Optica...",28.584485,77.220316,DC-001,15 kW,Self Service


**Data Cleaning**

In [28]:
# Drop duplicates
before = df.shape[0]
df.drop_duplicates(inplace=True)
after = df.shape[0]
print(f"Removed {before - after} duplicate rows.")

# Handle missing coordinates
missing_coords = df[df['latitude'].isnull() | df['longitude'].isnull()]
print(f"Rows with missing coordinates: {len(missing_coords)}")

# Option: Drop rows with missing lat/lon (if few)
df = df.dropna(subset=['latitude', 'longitude'])

# Handle missing region or type_power if possible
df['region'] = df['region'].fillna('Unknown')
df['service'] = df['service'].fillna('Unknown')

print("Cleaning done ✅")
df.info()

Removed 34 duplicate rows.
Rows with missing coordinates: 0
Cleaning done ✅
<class 'pandas.core.frame.DataFrame'>
Index: 168 entries, 0 to 200
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   region      168 non-null    object 
 1   address     168 non-null    object 
 2   aux_addres  168 non-null    object 
 3   latitude    168 non-null    float64
 4   longitude   168 non-null    float64
 5   type        168 non-null    object 
 6   power       168 non-null    object 
 7   service     168 non-null    object 
dtypes: float64(2), object(6)
memory usage: 11.8+ KB


In [29]:
# Save to a CSV file
cleaned_file_path = "/content/ev_charging_station_list_cleaned.csv"
df.to_csv(cleaned_file_path, index=False)

# Download to your local system
from google.colab import files
files.download(cleaned_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

This is for download of cleaned data


**Data loading** For the Electric car data clean

In [13]:
# Replace with your file path in Drive
file_path = "/content/ElectricCarData_Clean.csv"

# Load CSV
df = pd.read_csv(file_path)

# Quick look at data
print("Shape:", df.shape)
df.head()


Shape: (103, 14)


,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,C,5,30000
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,D,5,56440
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,D,5,68040
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,B,4,32997


**Data preprocessing**

In [14]:
# Standardize column names (lowercase, underscores)
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Strip extra spaces in object columns
for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].str.strip()

# Convert numeric columns
numeric_cols = ['accelsec', 'topspeed_kmh', 'range_km', 'efficiency_whkm',
                'fastcharge_kmh', 'seats', 'priceeuro']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

print("Preprocessing done ✅")
df.info()


Preprocessing done ✅
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   brand            103 non-null    object 
 1   model            103 non-null    object 
 2   accelsec         103 non-null    float64
 3   topspeed_kmh     103 non-null    int64  
 4   range_km         103 non-null    int64  
 5   efficiency_whkm  103 non-null    int64  
 6   fastcharge_kmh   98 non-null     float64
 7   rapidcharge      103 non-null    object 
 8   powertrain       103 non-null    object 
 9   plugtype         103 non-null    object 
 10  bodystyle        103 non-null    object 
 11  segment          103 non-null    object 
 12  seats            103 non-null    int64  
 13  priceeuro        103 non-null    int64  
dtypes: float64(2), int64(5), object(7)
memory usage: 11.4+ KB


**Data cleaning**

In [15]:
# Drop duplicates
before = df.shape[0]
df.drop_duplicates(inplace=True)
after = df.shape[0]
print(f"Removed {before - after} duplicate rows.")

# Handle missing values
df['brand'].fillna('Unknown', inplace=True)
df['model'].fillna('Unknown', inplace=True)
df['rapidcharge'].fillna('Unknown', inplace=True)
df['powertrain'].fillna('Unknown', inplace=True)
df['plugtype'].fillna('Unknown', inplace=True)
df['bodystyle'].fillna('Unknown', inplace=True)
df['segment'].fillna('Unknown', inplace=True)

# Fill numeric NaNs with median
for col in numeric_cols:
    df[col].fillna(df[col].median(), inplace=True)

# Remove unrealistic values
df = df[df['range_km'] > 0]
df = df[df['priceeuro'] > 0]

print("Cleaning done ✅")
df.describe()


Removed 0 duplicate rows.
Cleaning done ✅


/tmp/ipython-input-2568841284.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['brand'].fillna('Unknown', inplace=True)
/tmp/ipython-input-2568841284.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using '

,accelsec,topspeed_kmh,range_km,efficiency_whkm,fastcharge_kmh,seats,priceeuro
count,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000
mean,7.396117,179.194175,338.786408,189.165049,455.922330,4.883495,55811.563107
std,3.017430,43.573030,126.014444,29.566839,196.301274,0.795834,34134.665280
min,2.100000,123.000000,95.000000,104.000000,170.000000,2.000000,20129.000000
25%,5.100000,150.000000,250.000000,168.000000,305.000000,5.000000,34429.500000
50%,7.300000,160.000000,340.000000,180.000000,440.000000,5.000000,45000.000000
75%,9.000000,200.000000,400.000000,203.000000,555.000000,5.000000,65000.000000
max,22.400000,410.000000,970.000000,273.000000,940.000000,7.000000,215000.000000


**Downloading cleaned data**

In [16]:
# Save to a CSV file
cleaned_file_path = "/content/ElectricCarData_cleaned.csv"
df.to_csv(cleaned_file_path, index=False)

# Download to your local system
from google.colab import files
files.download(cleaned_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Data loading**  For dataset called EVIndia

In [17]:
# Path to dataset (adjust filename if needed)
file_path = '/content/EVIndia.csv'

# Load CSV
df = pd.read_csv(file_path)

# Quick preview
print("Shape of dataset:", df.shape)
df.head()

Shape of dataset: (12, 10)


,Car,Style,Range,Transmission,VehicleType,PriceRange(Lakhs),Capacity,BootSpace,BaseModel,TopModel
0,Tata Nexon EV,Compact SUV,312 Km/Full Charge,Automatic,Electric,17.40,5 Seater,350 L,XM,Dark XZ Plus LUX
1,Tata Tigor EV,Subcompact Sedan,306 Km/Full Charge,Automatic,Electric,13.64,5 Seater,316 L,XE,XZ Plus Dual Tone
2,Tata Nexon EV Max,Compact SUV,437 Km/Full Charge,Automatic,Electric,19.24,5 Seater,350 L,XZ Plus 3.3 kW,XZ Plus Lux 7.2 kW
3,MG ZS EV,Compact SUV,419 Km/Full Charge,Automatic,Electric,25.88,5 Seater,448 L,Excite,Exclusive
4,Hyundai Kona Electric,Compact SUV,452 Km/Full Charge,Automatic,Electric,23.98,5 Seater,na,Premium Dual Tone,HSE


**Data preprocessing**

In [35]:
# =========================
# 3️⃣ DATA PREPROCESSING
# =========================

# 1. Standardize column names
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Print column names after standardization to verify
print("\nColumns after standardization:", df.columns.tolist())

# 2. Clean specific columns if they exist in the dataset
if 'range' in df.columns:
    df['range'] = df['range'].astype(str).str.replace(' km/full charge', '', case=False).str.strip()

if 'capacity' in df.columns:
    df['capacity'] = df['capacity'].astype(str).str.replace(' seater', '', case=False).str.strip()

if 'bootspace' in df.columns:
    df['bootspace'] = df['bootspace'].astype(str).str.replace(' l', '', case=False).str.strip()
    df['bootspace'] = df['bootspace'].replace(['na', 'NA', 'NaN', 'nan'], np.nan)

# 3. Check missing values
print("\nMissing values per column:\n", df.isnull().sum())

# 4. Data types before conversion
print("\nData types before conversion:\n", df.dtypes)

# 5. Convert numerical columns (only if they exist)
numeric_columns = ['range', 'pricerange(lakhs)', 'capacity', 'bootspace']
for col in numeric_columns:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# 6. Clean text columns (only if they exist)
text_columns = ['car', 'style', 'transmission', 'vehicletype', 'basemodel', 'topmodel']
for col in text_columns:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()

print("\nPreprocessing complete ✅")
display(df.head())



Columns after standardization: ['region', 'address', 'aux_addres', 'latitude', 'longitude', 'type', 'power', 'service']

Missing values per column:
 region        0
address       0
aux_addres    0
latitude      0
longitude     0
type          0
power         0
service       0
dtype: int64

Data types before conversion:
 region         object
address        object
aux_addres     object
latitude      float64
longitude     float64
type           object
power          object
service        object
dtype: object

Preprocessing complete ✅


,region,address,aux_addres,latitude,longitude,type,power,service
0,NDMC,"Prithviraj Market, Rabindra Nagar, New Delhi- ...","Electric Vehicle Charger, Prithviraj Market, R...",28.600725,77.226252,DC-001,15 kW,Self Service
2,NDMC,"Outside RWA Park, Jor Bagh Market, Jor Bagh Co...","Electric Vehicle Charger, Outside RWA Park, Jo...",28.588303,77.217697,DC-001,15 kW,Self Service
3,NDMC,"Opposite Dory Pharmacy, Khanna Market, Aliganj...","Electric Vehicle Charger, Opposite Dory Pharma...",28.582654,77.220087,DC-001,15 kW,Self Service
4,NDMC,"Opposite Goel Opticals, Khanna Market, Aliganj...","Electric Vehicle Charger, Opposite Goel Optica...",28.584485,77.220316,DC-001,15 kW,Self Service
5,NDMC,"Dharma Marg, Block Y, Diplomatic Enclave, Malc...","Electric Vehicle Charger, Dharma Marg, Block Y...",28.602356,77.186618,DC-001,15 kW,Self Service


**Data cleaning**

In [19]:
# =========================
# 4️⃣ DATA CLEANING
# =========================

# Remove duplicates
before = df.shape[0]
df.drop_duplicates(inplace=True)
after = df.shape[0]
print(f"Removed {before - after} duplicate rows.")

# Handle missing numerical values (fill with median)
for col in numeric_columns:
    df[col].fillna(df[col].median(), inplace=True)

# Handle missing text values (fill with 'Unknown')
for col in text_columns:
    df[col] = df[col].replace('nan', np.nan)
    df[col].fillna('Unknown', inplace=True)

# Final check
print("\nData after cleaning:")
print(df.info())
print("\nSample cleaned data:")
df.head()


Removed 0 duplicate rows.

Data after cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   car                12 non-null     object 
 1   style              12 non-null     object 
 2   range              0 non-null      float64
 3   transmission       12 non-null     object 
 4   vehicletype        12 non-null     object 
 5   pricerange(lakhs)  12 non-null     float64
 6   capacity           0 non-null      float64
 7   bootspace          0 non-null      float64
 8   basemodel          12 non-null     object 
 9   topmodel           12 non-null     object 
dtypes: float64(4), object(6)
memory usage: 1.1+ KB
None

Sample cleaned data:


/tmp/ipython-input-4064770851.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/tmp/ipython-input-4064770851.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

,car,style,range,transmission,vehicletype,pricerange(lakhs),capacity,bootspace,basemodel,topmodel
0,Tata Nexon EV,Compact SUV,NaN,Automatic,Electric,17.40,NaN,NaN,XM,Dark XZ Plus LUX
1,Tata Tigor EV,Subcompact Sedan,NaN,Automatic,Electric,13.64,NaN,NaN,XE,XZ Plus Dual Tone
2,Tata Nexon EV Max,Compact SUV,NaN,Automatic,Electric,19.24,NaN,NaN,XZ Plus 3.3 kW,XZ Plus Lux 7.2 kW
3,MG ZS EV,Compact SUV,NaN,Automatic,Electric,25.88,NaN,NaN,Excite,Exclusive
4,Hyundai Kona Electric,Compact SUV,NaN,Automatic,Electric,23.98,NaN,NaN,Premium Dual Tone,HSE


**Downloading the dataset**

In [20]:
from google.colab import files

cleaned_file_path = "EVIndia_cleaned.csv"
df.to_csv(cleaned_file_path, index=False)
files.download(cleaned_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Data loading** For Cheapest electric cars database

In [21]:
file_path = '/content/Cheapestelectriccars-EVDatabase.csv'  # change filename
df = pd.read_csv(file_path)

# Overview
print("Shape of dataset:", df.shape)
df.head()

Shape of dataset: (180, 11)


,Name,Subtitle,Acceleration,TopSpeed,Range,Efficiency,FastChargeSpeed,Drive,NumberofSeats,PriceinGermany,PriceinUK
0,Opel Ampera-e,Battery Electric Vehicle | 58 kWh,7.3 sec,150 km/h,335 km,173 Wh/km,210 km/h,Front Wheel Drive,5,"€42,990",NaN
1,Renault Kangoo Maxi ZE 33,Battery Electric Vehicle | 31 kWh,22.4 sec,130 km/h,160 km,194 Wh/km,-,Front Wheel Drive,5,NaN,"£31,680"
2,Nissan Leaf,Battery Electric Vehicle | 36 kWh,7.9 sec,144 km/h,220 km,164 Wh/km,230 km/h,Front Wheel Drive,5,"€29,990","£25,995"
3,Audi e-tron Sportback 55 quattro,Battery Electric Vehicle | 86.5 kWh,5.7 sec,200 km/h,375 km,231 Wh/km,600 km/h,All Wheel Drive,5,NaN,"£79,900"
4,Porsche Taycan Turbo S,Battery Electric Vehicle | 83.7 kWh,2.8 sec,260 km/h,390 km,215 Wh/km,860 km/h,All Wheel Drive,4,"€186,336","£138,830"


**Data preprocessing**

In [22]:
# =========================
# 3️⃣ DATA PREPROCESSING
# =========================

# Standardize column names (lowercase, underscores)
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Check missing values
print("\nMissing values per column:\n", df.isnull().sum())

# Check data types before conversion
print("\nData types before conversion:\n", df.dtypes)

# Convert numeric columns to proper types
numeric_columns = [
    'acceleration', 'topspeed', 'range', 'efficiency',
    'fastchargespeed', 'numberofseats', 'priceingermany', 'priceinuk'
]

for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Clean text columns (strip spaces)
text_columns = ['name', 'subtitle', 'drive']
for col in text_columns:
    df[col] = df[col].astype(str).str.strip()

print("\nPreprocessing complete ✅")
df.head()



Missing values per column:
 name                0
subtitle            0
acceleration        0
topspeed            0
range               0
efficiency          0
fastchargespeed     0
drive               0
numberofseats       0
priceingermany     12
priceinuk          44
dtype: int64

Data types before conversion:
 name               object
subtitle           object
acceleration       object
topspeed           object
range              object
efficiency         object
fastchargespeed    object
drive              object
numberofseats       int64
priceingermany     object
priceinuk          object
dtype: object

Preprocessing complete ✅


,name,subtitle,acceleration,topspeed,range,efficiency,fastchargespeed,drive,numberofseats,priceingermany,priceinuk
0,Opel Ampera-e,Battery Electric Vehicle | 58 kWh,NaN,NaN,NaN,NaN,NaN,Front Wheel Drive,5,NaN,NaN
1,Renault Kangoo Maxi ZE 33,Battery Electric Vehicle | 31 kWh,NaN,NaN,NaN,NaN,NaN,Front Wheel Drive,5,NaN,NaN
2,Nissan Leaf,Battery Electric Vehicle | 36 kWh,NaN,NaN,NaN,NaN,NaN,Front Wheel Drive,5,NaN,NaN
3,Audi e-tron Sportback 55 quattro,Battery Electric Vehicle | 86.5 kWh,NaN,NaN,NaN,NaN,NaN,All Wheel Drive,5,NaN,NaN
4,Porsche Taycan Turbo S,Battery Electric Vehicle | 83.7 kWh,NaN,NaN,NaN,NaN,NaN,All Wheel Drive,4,NaN,NaN


**Data cleaning**

In [23]:
# =========================
# 4️⃣ DATA CLEANING
# =========================

# Remove duplicates
before = df.shape[0]
df.drop_duplicates(inplace=True)
after = df.shape[0]
print(f"Removed {before - after} duplicate rows.")

# Handle missing numerical values (fill with median)
for col in numeric_columns:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].median(), inplace=True)

# Handle missing text values (fill with 'Unknown')
for col in text_columns:
    df[col] = df[col].replace('nan', np.nan)
    df[col].fillna('Unknown', inplace=True)

# Final check
print("\nData after cleaning:")
print(df.info())
print("\nSample cleaned data:")
df.head()


Removed 5 duplicate rows.

Data after cleaning:
<class 'pandas.core.frame.DataFrame'>
Index: 175 entries, 0 to 179
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             175 non-null    object 
 1   subtitle         175 non-null    object 
 2   acceleration     0 non-null      float64
 3   topspeed         0 non-null      float64
 4   range            0 non-null      float64
 5   efficiency       0 non-null      float64
 6   fastchargespeed  0 non-null      float64
 7   drive            175 non-null    object 
 8   numberofseats    175 non-null    int64  
 9   priceingermany   0 non-null      float64
 10  priceinuk        0 non-null      float64
dtypes: float64(7), int64(1), object(3)
memory usage: 16.4+ KB
None

Sample cleaned data:


/tmp/ipython-input-2035105538.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/tmp/ipython-input-2035105538.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

,name,subtitle,acceleration,topspeed,range,efficiency,fastchargespeed,drive,numberofseats,priceingermany,priceinuk
0,Opel Ampera-e,Battery Electric Vehicle | 58 kWh,NaN,NaN,NaN,NaN,NaN,Front Wheel Drive,5,NaN,NaN
1,Renault Kangoo Maxi ZE 33,Battery Electric Vehicle | 31 kWh,NaN,NaN,NaN,NaN,NaN,Front Wheel Drive,5,NaN,NaN
2,Nissan Leaf,Battery Electric Vehicle | 36 kWh,NaN,NaN,NaN,NaN,NaN,Front Wheel Drive,5,NaN,NaN
3,Audi e-tron Sportback 55 quattro,Battery Electric Vehicle | 86.5 kWh,NaN,NaN,NaN,NaN,NaN,All Wheel Drive,5,NaN,NaN
4,Porsche Taycan Turbo S,Battery Electric Vehicle | 83.7 kWh,NaN,NaN,NaN,NaN,NaN,All Wheel Drive,4,NaN,NaN


**Downloading the dataset**

In [24]:
from google.colab import files

# Save cleaned file
cleaned_file_path = "cheapest_electric_cars_dataset_cleaned.csv"
df.to_csv(cleaned_file_path, index=False)

# Download
files.download(cleaned_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>